In [43]:
import pandas as pd
import sqlite3


In [44]:
import pandas as pd
import sqlite3
import glob

# Correct folder path (ensure it does NOT end with `\` to avoid double slashes)
folder_path = r"C:\Users\doris\Desktop\Data management\archive\dataset"

# Get all CSV file paths in the folder
csv_files = glob.glob(folder_path + "/*.csv")  # Uses `/*.csv` to find all CSVs

# Check if files are found
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {folder_path}")

# Connect to SQLite database (in-memory)
conn = sqlite3.connect(":memory:")

# Loop through each CSV file and load into SQLite
for file in csv_files:
    file_name = file.split("\\")[-1].replace(".csv", "")  # Extracts file name (without extension)
    clean_name = file_name.replace(" ", "_")  # Replaces spaces with underscores to prevent SQL errors

    print(f"Loading: {clean_name}")  # Print which file is being loaded
    
    # Read CSV with error handling
    try:
        df = pd.read_csv(file, low_memory=False)
    except Exception as e:
        print(f"❌ Error loading {file_name}: {e}")
        continue  # Skip this file and continue with the next one

    # Store DataFrame in SQLite with the cleaned table name
    df.to_sql(clean_name, conn, index=False, if_exists="replace")

print("✅ All CSV files successfully loaded!")

# Query an example table (Change `table_name` to any loaded table)
query = f"SELECT * FROM Amazon_Sale_Report LIMIT 5;"  # Uses the last loaded table as default
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)



Loading: Amazon_Sale_Report
Loading: International_sale_Report
Loading: Sale_Report
✅ All CSV files successfully loaded!


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,None,0,Easy Ship,None
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,0,Easy Ship,None
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,1,None,None
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,None,0,Easy Ship,None
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,None,0,None,None


In [45]:
# Connect 3 tables together 
query = """
SELECT asr.*, sr.*, isr.*
FROM Amazon_Sale_Report asr 
LEFT JOIN Sale_Report sr ON asr.sku = sr.[SKU Code] 
LEFT JOIN International_sale_Report isr ON asr.sku = isr.sku;
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)



,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,7911.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,SET389,SET389-KR-NP-S,S,1.00,1000.00,1000.00
1,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,13607.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,SET389,SET389-KR-NP-S,S,1.00,1000.00,1000.00
2,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,27846.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,SET389,SET389-KR-NP-S,1.00,1000.00,1000.00,60.00
3,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,33711.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,SET389,SET389-KR-NP-S,1.00,1000.00,1000.00,60.00
4,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,NaN,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828633,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,33678.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,J0012,J0012-SKD-XS,1.00,1387.50,1388.00,119.00
828634,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,7990.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,J0003,J0003-SET-S,S,1.00,737.50,738.00
828635,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,13686.0,01-29-22,Jan-22,MULBERRIES BOUTIQUE,J0003,J0003-SET-S,S,1.00,737.50,738.00
828636,128974,408-7436540-8728312,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0003,J0003-SET-S,Set,...,27930.0,MULBERRIES BOUTIQUE,01-29-22,Jan-22,J0003,J0003-SET-S,1.00,737.50,738.00,15.00


In [46]:
# Customer & Market Insights: Who are the top 10 international customers by purchase volume?
query = """
WITH a AS (
    SELECT asr.amount, isr.customer
    FROM Amazon_Sale_Report asr 
    LEFT JOIN International_sale_Report isr ON asr.sku = isr.sku
)
SELECT customer, SUM(amount) AS sum_sales
FROM a 
GROUP BY customer
ORDER BY sum_sales DESC
LIMIT 10;

"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)


,customer,sum_sales
0,Feb-22,44928420.64
1,Oct-21,40310380.30
2,Mar-22,30851080.91
3,None,30010070.58
4,MULBERRIES BOUTIQUE,29687323.97
5,Jan-22,24857548.04
6,Sep-21,24027371.16
7,Nov-21,20123376.58
8,Dec-21,14171149.52
9,Jul-21,13135973.41


In [47]:
#Customer & Market Insights: Top 10 cities order from Amazon sorted by amount?
query ='''
SELECT "ship-city", SUM(amount) AS sales_amount
FROM Amazon_Sale_Report
GROUP BY "ship-city"
ORDER BY sales_amount DESC
LIMIT 10;

'''
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,ship-city,sales_amount
0,BENGALURU,6849664.99
1,HYDERABAD,4946032.82
2,MUMBAI,3704461.80
3,NEW DELHI,3613874.78
4,CHENNAI,3098745.74
5,PUNE,2338518.18
6,KOLKATA,1414978.87
7,GURUGRAM,1221618.74
8,THANE,1004355.29
9,LUCKNOW,933926.34


In [48]:
# Operational Efficiency: Calculate the number of canceled or returned orders
query = """
select [Courier Status],count([Order ID])
from Amazon_Sale_Report 
group by ([Courier Status])
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)


,Courier Status,count([Order ID])
0,None,6872
1,Cancelled,5935
2,Shipped,109487
3,Unshipped,6681


In [49]:
# Operational Efficiency: Which fulfillment method (e.g., FBA vs. self-fulfilled) has the lowest shipping failure rate?
query = """
select [Fulfilment],count([Order ID])
from Amazon_Sale_Report 
group by [Fulfilment]
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,Fulfilment,count([Order ID])
0,Amazon,89698
1,Merchant,39277


In [50]:
# Change the date data type
cursor = conn.cursor()
query = """
UPDATE Amazon_Sale_Report
SET Date = '20' || substr(Date, -2, 2) || '-' || substr(Date, 1, 2) || '-' || substr(Date, 4, 2);



"""
# Execute the update query
cursor.execute(query)

# Commit changes and close the connection
conn.commit()

print("Date format updated successfully!")

Date format updated successfully!


In [53]:
# Check date data type
query = """
SELECT Date
FROM Amazon_Sale_Report
WHERE Date > '2022-02-22';
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,Date
0,2022-04-30
1,2022-04-30
2,2022-04-30
3,2022-04-30
4,2022-04-30
...,...
128970,2022-05-31
128971,2022-05-31
128972,2022-05-31
128973,2022-05-31


In [57]:
#Sales Performance: What is the total gross amount for sales in April 2022?  I changed to April, since there is no data for Jan
query = """
SELECT SUM(amount)
FROM Amazon_Sale_Report
WHERE Date BETWEEN '2022-04-01' AND '2022-04-30';

"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)


,SUM(amount)
0,28838708.32


In [60]:
#Sales Performance: Which top 5 SKUs generated the highest revenue in April 2022?   I changed to April, since there is no data for Jan and Feb
query = """
SELECT SKU, SUM(amount) AS total_sales
FROM Amazon_Sale_Report
WHERE Date BETWEEN '2022-04-01' AND '2022-04-30'
GROUP BY SKU
ORDER BY total_sales DESC
limit (5)
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,SKU,total_sales
0,J0230-SKD-M,274978.00
1,J0230-SKD-S,261601.86
2,SET268-KR-NP-XL,229064.00
3,SET268-KR-NP-L,222558.00
4,SET268-KR-NP-S,218962.48


In [64]:
#Sales Performance: Will there be a higher sales volume on holidays and weekends compared to working days?
query = """
WITH b AS (
    SELECT 
        Date,
        amount,
        CASE 
            WHEN strftime('%w', Date) IN ('0', '6') THEN 'Weekend' 
            ELSE 'Weekday' 
        END AS Day_Type
    FROM Amazon_Sale_Report
)
SELECT 
    Day_Type, 
    SUM(amount) / COUNT(DISTINCT Date) AS ave_sales_per_day
FROM b
GROUP BY Day_Type;


"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,Day_Type,ave_sales_per_day
0,Weekday,849598.145077
1,Weekend,898799.956538


In [68]:
#Inventory & Product Analysis: Which product has never been sold in any report?
query = """
SELECT SKU, SUM(Qty) AS total_quantity
FROM Amazon_Sale_Report
GROUP BY SKU
HAVING SUM(Qty) = 0
"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,SKU,total_quantity
0,AN204-PURPLE-XL,0
1,AN210-NAVY-M,0
2,BL091-XL,0
3,BL100-XXL,0
4,BL104-XL,0
5,BTM002-B-XXL,0
6,BTM027-NP-L,0
7,BTM032-NP-M,0
8,J0024-KR-XXXL,0
9,J0034-SET-S,0


In [69]:
#Inventory & Product Analysis: Which size is most popular for shipped items?
query = """
select Size, count(*) as sales_item
FROM Amazon_Sale_Report
group by Size


"""
result = pd.read_sql(query, conn)

# Display results in Jupyter Notebook
from IPython.display import display
display(result)

,Size,sales_item
0,3XL,14816
1,4XL,427
2,5XL,550
3,6XL,738
4,Free,378
5,L,22132
6,M,22711
7,S,17090
8,XL,20876
9,XS,11161


In [13]:
# Close connection
conn.close()
